# 政府統計データを使った機械学習分析

国勢調査等の政府統計データを用いて市区町村を分類し、GeoPandasとPlotly Expressを使った視覚化を試みる。<br>

## Part1 Data Downloading

【参照資料】
e-Statsからのデータダウンロード：<br>
https://note.nkmk.me/python-e-stat-api-download/

### ライブラリのインポート

In [1]:
import pandas as pd
import numpy as np
import os
import re
import json
import time
import copy
import urllib.parse
import urllib.request

### 利用可能データのリストを作成
政府統計の総合窓口(e-Stat: https://www.e-stat.go.jp/)

In [2]:
def get_list(param, form='json'):
    ''' 作成されたAPIリクエストの送信とレスポンスの処理 '''
    
    # Create URL to get information
    url = 'http://api.e-stat.go.jp/rest/3.0/app/'
    if form == 'xml':
        url += 'getStatsList?'
    else:  # json
        url += 'json/getStatsList?'
    
    url += urllib.parse.urlencode(param)
    with urllib.request.urlopen(url) as response:
        return response.read().decode()

In [3]:
def get_list_table_inf(param):
    d = json.loads(get_list(param, 'json'))
    return d['GET_STATS_LIST']['DATALIST_INF']['TABLE_INF']

In [4]:
def save_list(param, dir_path='.', filename='stats_list', form='csv',
              replace=True, sep='_', atmark='', dollar='val', **kwargs):
    
    ''' フォーマットの変換とリストの保存 '''

    path = os.path.join(dir_path, filename + '.' + form)
    if form == 'csv':
        # Load and decode json
        l = get_list_table_inf(param)
        if replace:
            # Convert json to pandas.DataFrame            
            df = pd.io.json.json_normalize(l, sep=sep)
            # Replace some characters so that query method can be used.            
            df.columns = [s.replace('@', atmark).replace('$', dollar) for s in df.columns]
            # Save as a csv file            
            df.to_csv(path, index=False)
        else:
            # Convert json to pandas.DataFrame            
            pd.io.json.json_normalize(l).to_csv(path, index=False)
    elif form == 'json':
        # Load and decoding json        
        d = json.loads(get_list(param, 'json'))
        with open(path, 'w') as f:
            # Convert into json strings            
            json.dump(d, f, **kwargs)
    else:  # xml
        with open(path, 'w') as f:
            # Load and save as a xml file            
            f.write(get_list(param, form))

APIリクエストに必要なID番号をjsonファイルとして、予め保存。e-statsのサイトから無料で申請可能。

In [5]:
with open('setting/app_id.json') as f:
    p_id = json.load(f)

利用可能統計データのリストをダウンロードし、csvファイルとして保存。

In [6]:
save_list(p_id, 'download', 'all_stats_list')

<br>
<br>

作成されたリストから、市区町村単位で集計されたデータのみをフィルターにかけ、データ名を修正して新たなcsvファイルとして保存。

In [9]:
stats_df = pd.read_csv('download/all_stats.csv')
Field_List = ['id','MAIN_CATEGORY_val','STATISTICS_NAME','SURVEY_DATE','TITLE','TITLE_val','OVERALL_TOTAL_NUMBER']
stats_df_city = stats_df[stats_df.COLLECT_AREA == '市区町村'][Field_List]
is_title_null = pd.isnull(stats_df_city.TITLE)
stats_df_city.loc[is_title_null,'TITLE'] = stats_df_city.loc[is_title_null,'TITLE_val']
stats_df_city = stats_df_city.drop(['TITLE_val'], axis=1)
stats_df_city.loc[:,'id'] = stats_df_city['id'].apply(str).str.rjust(10,'0')
stats_df_city.to_csv('download/all_stats_city.csv',index=False)

### 市区町村データの一部をダウンロード

In [10]:
def get_data(param, form, path):
    '''
    APIリクエストの作成とレスポンスの処理。
    10万件のレコード数が一度にダウンロードできる上限のため、それ以上のレコード数を持つデータの場合は、
    複数回に分けてダウンロードし、appendする必要がある。
    '''
    
    # Create URL to get information
    url = 'http://api.e-stat.go.jp/rest/3.0/app/'
    url += 'getSimpleStatsData?'
    
    # Open URL and get data or list.    
    url_prm = url + urllib.parse.urlencode(param)
    with urllib.request.urlopen(url_prm) as response:
        data = response.read().decode()
    
    string = re.split(',|\n',data.replace('"',''))[0:100]
    total_number = int(string[string.index('TOTAL_NUMBER') + 1])
    num_limit = int(1e5)
    if total_number < num_limit:
        with open(path, 'w') as f:
            f.write(data)    
    else:
        param['sectionHeaderFlg'] = 2
        for i in range(int(total_number // num_limit)):
            string = re.split(',|\n',data.replace('"',''))[0:100]
            param['startPosition'] = (i + 1)*num_limit + 1
            url_prm = url + urllib.parse.urlencode(param)
            with urllib.request.urlopen(url_prm) as response:
                data += response.read().decode()
        with open(path, 'w') as f:
            f.write(data)

In [11]:
def save_data(param, stats_data_id=None, dir_path='.', filename=None,
              form='csv', section_header=False, skip=True, **kwargs):
    
    p = param.copy()
    if stats_data_id:
        p['statsDataId'] = stats_data_id

    if not filename:
        filename = p['statsDataId']

    path = os.path.join(dir_path, filename + '.' + form)

    if section_header:
        p['sectionHeaderFlg'] = 1
    else:
        p['sectionHeaderFlg'] = 2

    if os.path.exists(path) & skip:
        print('skip {} ({} already exists)'.format(p['statsDataId'], path))
    else:
        print('download {} to {}'.format(p['statsDataId'], path))
        get_data(p, 'csv', path)

In [12]:
def save_data_multi(param, ids, dir_path='.', filename_words=None, sep='_', form='csv',
                    section_header=False, skip=True, interval_time_sec=1, **kwargs):

    if filename_words:
        for i, *words in zip(ids, *filename_words):
            time.sleep(interval_time_sec)
            save_data(param, i, dir_path, sep.join([str(word) for word in words]),
                      form, section_header, skip, **kwargs)
    else:
        for i in ids:
            time.sleep(interval_time_sec)
            save_data(param, i, dir_path, i, form, section_header, skip, **kwargs)

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

以下の条件に該当するデータをダウンロード。<br>
１．市区町村毎のデータ分析を目的とするため、県あるいは地域単位のデータ以外。差し当たり、レコード数でフィルターをかける。<br>
２．古い版のデータを避けるため、リストを見て調査年が2009年度と2013年度のデータを除外。<br>
３．農林水作業データは扱い方も取り敢えず無視。<br>

時間が非常にかかるため、中身を見ながら使えそうなものだけ拾うのがいいかも知れません。

In [ ]:
stats_df = pd.read_csv('download/all_stats_city.csv')
MinRecords = 4e4
MaxRecords = 1e7
stats_df_city = stats_df_city.loc[(stats_df_city.MAIN_CATEGORY_val != '農林水産業') &
                                 (stats_df_city['SURVEY_DATE'].str.contains('2009') == False) &
                                 (stats_df_city['SURVEY_DATE'].str.contains('2013') == False) & 
                                 (stats_df_city['OVERALL_TOTAL_NUMBER'] >= MinRecords) & 
                                 (stats_df_city['OVERALL_TOTAL_NUMBER'] <= MaxRecords)]

id_list = stats_df_city['id'].values.tolist()
dir_path = 'download/stats_city'
save_data_multi(p_id, id_list, dir_path, section_header=True)

### データのスクリーニング

In [13]:
def read_stats_csv(path_input,file):
    ''' ヘッダーの行数を探した後に、csvファイルを読み込む。 '''
    path_join = os.path.join(path_input,file)
    with open(path_join,"r") as file_text:
        skiprows = file_text.read(2000).replace('"','').split('\n').index('VALUE') + 1

    df = pd.read_csv(path_join, encoding="shift-jis", skiprows=skiprows)
    return df

In [14]:
def filter_by_num_records(input_path,num_min_rec=1800):
    ''' レコード数を元に使えそうなファイルを抽出し、リストに保存。 '''
    list_error = []
    list_code = []
    list_title = []
    list_filename = []
    list_area_code_num = []
    for file in os.listdir(input_path):
        try:
            df = read_stats_csv(input_path,file)
            num_area_code = len(df.area_code.unique())
            list_area_code_num.append(num_area_code)
            list_filename.append(file)
            if num_area_code > num_min_rec:
                flag_code = stats_df_city['id'].apply(int) == int(file.split('.csv')[0])
                list_title.append(stats_df_city.loc[flag_code,:].TITLE.values[0])
                list_code.append(file)
        except:
            list_error.append(file)
            pass;
    return list_code, list_title, list_area_code_num, list_filename, list_error

In [16]:
dir_path = 'download/stats_city'
list_code,list_title,_,_,_ = filter_by_num_records(dir_path)
dict_use = dict(zip(list_code,list_title))
df_field_use = pd.DataFrame.from_dict(dict_use, orient='index')
df_field_use.to_csv('./df_field_use.csv')